[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/generation/generative-qa/openai-ml-qa/00-build-index.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/generation/generative-qa/openai-ml-qa/00-build-index.ipynb)

# Index Init

We use this notebook to create embeddings with OpenAI and push the embeddings and metadata to Pinecone. Required installs for this notebook are:

In [1]:
!pip install -qU openai tiktoken pinecone-client datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.9 MB/s eta 0:00:00


## Data Preparation

We start by downloading the dataset from Hugging Face *Datasets*:

In [2]:
from datasets import load_dataset

data = load_dataset('jamescalam/ml-qa', split='train')
data

Dataset({
    features: ['docs', 'category', 'thread', 'href', 'question', 'context', 'marked'],
    num_rows: 6165
})

In [3]:
data[100]

{'docs': 'huggingface',
 'category': 'Beginners',
 'thread': 'Training stops when I try Fine-Tune XLSR-Wav2Vec2 for low-resource ASR',
 'href': 'https://discuss.huggingface.co/t/training-stops-when-i-try-fine-tune-xlsr-wav2vec2-for-low-resource-asr/8981',
 'question': 'Hi,\nI’m learning Wav2Vec2 according the blog link:\n  \n\n      huggingface.co\n  \n\n  \n    \n\nFine-Tune XLSR-Wav2Vec2 for low-resource ASR with 🤗 Transformers 1\n\n\n\n  \n\n  \n    \n    \n  \n\n  \n\n\nAnd I download the ipynb file and try run it locally.\nFine_Tune_XLSR_Wav2Vec2_on_Turkish_ASR_with_🤗_Transformers.ipynb\nAll looks file but when I run trainer.train(), it seems stop after a while, and it generate some log files under the folder wav2vec2-large-xlsr-turkish-demo, I send the screen shot to you as following:\n\n2021-08-05 17-05-36 的屏幕截图1063×410 35 KB\n\nI don’t know how to open the file events.out.tfevents.1628152300.tq-sy.129248.2, what’s the problem and how can I debug of it? please help.\nThanks a lo

When storing the original plaintext (and other metadata) of our data, we can either store them in Pinecone as indexed or non-indexed metadata — or elsewhere.

Storing in Pinecone can make the system simpler as we are then querying a single location. However, there are limitations on metadata size. For *indexed* metadata this is 5KB of metadata, and for *non-indexed* metadata this is 40KB, [see here for more info](https://docs.pinecone.io/docs/limits#:~:text=Max%20metadata%20size%20per%20vector,key%20from%20the%20metadata%20payload.).

First, let's check if we can fit our data within the larger 40KB limit.

In [4]:
from sys import getsizeof
import json

limit = 0

for record in data:
    size = getsizeof(json.dumps(record))
    if size > 40_000:
        limit += 1

print(f"Over 40KB: {(limit/len(data)*100)}%")

Over 40KB: 0.0%


Fortunately our data is within this limit (40KB is a lot), so we don't need to remove anything to fit with Pinecone limits, but what about OpenAI limits?

The current *token limit* for `text-embedding-ada-002` (the embedding model we will be using is **8191** tokens. However, later we will be using the `text-davinci-003` model for generation of answers, this model has a max context window of **~4K** tokens. We'll also want to feed in a few records here so we should limit ourselves to something closer to ~1000 tokens as a max limit.

We calculate the number of tokens using the `tiktoken` tokenizer like so:

In [11]:
import tiktoken

tokenizer = tiktoken.encoding_for_model('text-davinci-003')
tokenizer

<Encoding 'p50k_base'>

Define token counting function:

In [12]:
# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

Now let's filter out records that are longer than the ~2K.

In [21]:
data = data.filter(
    lambda x: 0 if tiktoken_len(
      '\n\n'.join([x['thread'], x['question'], x['context']])
    ) > 2_000 else 1
)
data

Filter:   0%|          | 0/6165 [00:00<?, ? examples/s]

Dataset({
    features: ['docs', 'category', 'thread', 'href', 'question', 'context', 'marked'],
    num_rows: 5947
})

We've dropped ~200 rows. In a real-world scenario you may want to simply limit or chunk records, but for this example simply dropping the rows will do.

Now let's move on to preparing the text data and building our embeddings.

## Text Prep and Embeddings

To store as much information as possible in each record, it may make sense to format each record as something like:

```
Thread title: <thread>

Question asked: <question>

Given answer: <context>
```

We will create this format for each record and store in a new `text` variable.

In [22]:
text = [
    f"Thread title: {x['thread']}\n\n"+
    f"Question asked: {x['question']}\n\n"+
    f"Given answer: {x['context']}" for x in data
]
print(text[100])

Thread title: Extremely confusing or non-existent documentation about the Seq2Seq trainer

Question asked: I’ve been trying to train a model to translate database metadata + human requests into valid SQL.
Initially, I used a wiki SQL base + a custom pytorch script (worked fine) but I decided I want to train my own from scratch and I’d better go with the “modern” method of using a trainer.
The code I currently have is:
        self.tokenizer = T5Tokenizer.from_pretrained("t5-small")
        self.model = T5ForConditionalGeneration.from_pretrained("t5-small")
        print('Creating datasets')
        train_dataset = Dataset.from_dict({
            'request': [x['prompt'] for x in data[:int(len(data) * 0.8)]],
            'label': [x['completion'] for x in data[:int(len(data) * 0.8)]]
        })
        eval_dataset = Dataset.from_dict({
            'request': [x['prompt'] for x in data[int(len(data) * 0.8):]],
            'label': [x['completion'] for x in data[int(len(data) * 0.8):]]
  

The text isn't always going to be perfect, but we'll see that the embedding model doesn't have any issues with this. Now let's initialize the embedding model and begin building the embeddings.

### Embedding with OpenAI

We begin by initializing the embedding model. For this we need [OpenAI API keys](https://beta.openai.com/signup).

In [23]:
import os
import openai

# get API key from top-right dropdown on OpenAI website
openai.api_key = os.getenv("OPENAI_API_KEY") or "OPENAI_API_KEY"

openai.Engine.list() # check we have authenticated

<OpenAIObject list at 0x7f301b6ddf40> JSON: {
  "data": [
    {
      "created": null,
      "id": "babbage",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "davinci",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-davinci-edit-001",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "babbage-code-search-code",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-similarity-babbage-001",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "code-davinci-edit-001",
      "object": "engine",
      

The `openai.Engine.list()` function should return a list of models that we can use. One of those is `text-embedding-ada-002` that we will use for creating embeddings like so:

In [24]:
model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=model
)

In the response `res` we will find a JSON-like object containing our new embeddings within the `'data'` field.

In [25]:
res.keys()

dict_keys(['object', 'data', 'model', 'usage'])

Inside `'data'` we will find two records, one for each of the two sentences we just embedded. Each vector embedding contains `1536` dimensions (the output dimensionality of the `text-embedding-ada-002` model.

In [26]:
len(res['data'])

2

In [27]:
len(res['data'][0]['embedding']), len(res['data'][1]['embedding'])

(1536, 1536)

We will apply this same embedding logic when indexing all of our data in the Pinecone vector database soon.

## Building a Pinecone Index

We need a vector index to store the vector embeddings and enable a fast and scalable search through them. For this we use the Pinecone vector database.

To use this we need a [free Pinecone API key](https://app.pinecone.io).

Once ready, we initialize our index like so:

In [28]:
import pinecone

index_name = 'openai-ml-qa'

# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(
    api_key="PINECONE_API_KEY",
    environment="YOUR_ENV"  # find next to API key in console
)
# check if 'openai-ml-qa' already exists (only create index if not)
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='cosine',
        # we may want to filter by docs (e.g. pytorch vs tensorflow)
        metadata_config={'indexed': ['docs']}
    )
# connect to index
index = pinecone.Index(index_name)

We added `metadata_config={'indexed': ['docs']}` to fit anything that we don't plan on using as filterable fields in the *non-indexed* category. By doing this we optimize our index space, keeping the index more efficient.

Now we begin populating the index.

When adding records to Pinecone we need three items in a tuple format:

```
(id, vector, metadata)
```

All IDs must be unique, our vectors will be built by OpenAI, and the metadata is a dictionary of the information for each record (`'href'`, `'question'`, etc).

We will create our vector embeddings and add the records to Pinecone in batches of `128`. This is to avoid trying to push too much data into single API requests.

In [29]:
from tqdm.auto import tqdm  # this is our progress bar

batch_size = 128  # process everything in batches of 128
for i in tqdm(range(0, len(text), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(text))
    # get batch of metadata, text, and IDs
    meta_batch = [data[x] for x in range(i,i_end)]
    text_batch = text[i:i_end]
    ids_batch = [str(n) for n in range(i, i_end)]
    # create embeddings
    res = openai.Embedding.create(input=text_batch, engine=model)
    embeds = [record['embedding'] for record in res['data']]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/47 [00:00<?, ?it/s]

We can check that everything has been upserted with `index.describe_index_stats()`

In [30]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5947}},
 'total_vector_count': 5947}

We have `5947` vectors (and their respective metadata) added to the index as expected.

With that our index has been built and we can move on to the next stage of querying in [01-making-queries.ipynb](https://github.com/pinecone-io/examples/blob/master/generation/generative-qa/openai-ml-qa/01-making-queries.ipynb).